In [1]:
%load_ext autoreload
%autoreload 2

## Multiple Choice Question Answer

In [2]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

tqdm.pandas()

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
train_df = pd.read_csv("data/real_labels_1.csv", index_col= 0 )
train_df_2 = pd.read_csv("data/real_labels_2.csv", index_col=0)
train_df_3 = pd.read_csv("data/real_labels_3.csv", index_col=0)

train_df = pd.concat([train_df, train_df_2, train_df_3])
train_df = train_df.drop_duplicates("originalTweet")

train_df['label'] = train_df['event'].map({
    "no": 0,
    "yes": 1,
    "penalty" : 1,
    "goal" : 1
})

train_df = train_df.sample(frac = 1., replace=False)\
    .dropna()

val_df = train_df.iloc[20_000:]
train_df = train_df.iloc[0:1_000]

In [5]:

train, test = train_test_split()

100%|██████████| 16/16 [00:08<00:00,  1.94it/s]


In [6]:
model =  AutoModel.from_pretrained("vinai/bertweet-base", cache_dir = '/Data')

for p in model.parameters():
    p.requires_grad = False
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", cache_dir = '/Data')

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
tokens = tokenizer(
    train_df['originalTweet'].tolist(),
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=64
)

vectors = model(**tokens)

In [ ]:
v = vectors.last_hidden_state.numpy().reshape(1_000, -1)

In [ ]:
pca = PCA(n_components=10)
X = pca.fit_transform(v)

In [ ]:

y = train_df['label']

# Define logistic regression model with class weights
clf = LogisticRegression()
clf.fit(X, y)

# K-Fold Cross Validation
kfold = StratifiedKFold(n_splits=100, shuffle=True, random_state=42)
fold_accuracies = []

df = pd.concat(train.values())

y = df['EventType']


for train_idx, test_idx in kfold.split(df, y):
    # Train-test split for the current fold
    X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print(len(X_test))
    
    tokens = tokenizer(
        X_test['Tweet'].tolist(),
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=64
    )

    vectors = model(**tokens)\
        .last_hidden_state\
        .numpy()\
        .reshape(len(X_test), -1)

    test_v = pca.fit_transform(vectors)
        
    X_test['proba_event'] = clf.predict_proba(test_v)[:, 1]
    
    # Apply threshold for classification
    X_test['is_event'] = (X_test['proba_event'] > 0.5).astype(int)
    
    # Aggregate predictions by MatchID and PeriodID
    pred_df = X_test.groupby(['MatchID', 'PeriodID'], as_index=False).agg({
        'is_event': np.max,
    })

    # Merge back to original test_df for evaluation
    merged_df = pd.merge(
        X_test,
        pred_df,
        on=['MatchID', 'PeriodID']
    )
    
    # Calculate accuracy
    acc = accuracy_score(merged_df['EventType'], merged_df['is_event_y'])
    fold_accuracies.append(acc)
    print(f"Fold Accuracy: {acc}")

# Calculate and print overall accuracy
mean_accuracy = np.mean(fold_accuracies)
print(f"Mean Cross-Validation Accuracy: {mean_accuracy}")

: 

In [109]:

X_i = vectorizer.transform(df['Tweet'])
df['proba_event'] = clf.predict_proba(X_i)[:,1]

# thresholds = np.arange(0.1, 1.0, 0.1)
# for t in thresholds:
df['is_event'] = (df['proba_event'] >0.99).astype(int)

pred_df = df.groupby(['MatchID', "PeriodID"], as_index=False).agg({
    "is_event": np.max,
})

merged_df = pd.merge(
    df,
    pred_df,
    on=['MatchID', 'PeriodID']
)
acc = accuracy_score(merged_df['EventType'], merged_df['is_event_y'])
print(acc)

0.45897839662505413


/tmp/ipykernel_702876/729903821.py:8: FutureWarning: The provided callable <function max at 0x7fe14c01d120> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  pred_df = df.groupby(['MatchID', "PeriodID"], as_index=False).agg({


In [92]:
merged_df['EventType'].value_counts()

EventType
1    471041
0    399610
Name: count, dtype: int64

In [93]:
merged_df['is_event_y'].value_counts()

is_event_y
0    732510
1    138141
Name: count, dtype: int64